In [ ]:
# DO NOT modify this cell.
filename = "Buchtel.pgm"
verticalSeam2Remove = 10
horizontalSeam2Remove = 8

# Yes, you can create your own test cases and you should. Do it in a new cell.

In [ ]:
# File operation functions

import numpy as np
import os

def strip_trailing_whitespace(input_file, output_file):
    with open(input_file, 'r') as f_in:
        with open(output_file, 'w') as f_out:
            for line in f_in:
                stripped_line = line.strip()
                f_out.write(stripped_line + '\n')

def load_image_as_numpy_array(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Check if the file is in a supported format
    file_format = lines[0].strip()
    if file_format not in ('P2', 'P3'):
        raise ValueError(f"Unsupported file format: {file_format}")

    # Extract image dimensions and maximum pixel value
    width, height = map(int, lines[2].split())
    max_value = int(lines[3])

    # Extract pixel values based on file format
    pixel_values = []
    if file_format == 'P2':  # PGM format (grayscale)
        for line in lines[4:]:
            pixel_values.extend(map(int, line.split()))
    elif file_format == 'P3':  # PPM format (RGB)
        for line in lines[4:]:
            values = line.split()
            for i in range(0, len(values), 3):
                r = int(values[i])
                g = int(values[i+1])
                b = int(values[i+2])
                pixel_values.append((r, g, b))
    else:
        raise ValueError(f"Unsupported file format: {file_format}")

    # Convert pixel values to a numpy array
    if file_format == 'P2':  # PGM format (grayscale)
        image_np = np.array(pixel_values).reshape(height, width)
    elif file_format == 'P3':  # PPM format (RGB)
        image_np = np.array(pixel_values).reshape(height, width, 3)

    return image_np, max_value

def save_image(image_filepath, image_np, max_value, v, h):
    """Save a numpy array as a PGM or PPM file based on the image array shape."""
    base_filename, ext = os.path.splitext(os.path.basename(image_filepath))
    output_directory = 'test/experimental/'
    output_filename = f"{output_directory}{base_filename}_processed_{v}_{h}{ext}"

    # Determine file format based on image array shape
    if image_np.ndim == 2:  # Grayscale image (PGM)
        file_format = "P2"
    elif image_np.ndim == 3 and image_np.shape[2] == 3:  # RGB image (PPM)
        file_format = "P3"
    else:
        raise ValueError("Unsupported image array shape")

    # Create the output directory if it does not exist
    os.makedirs(output_directory, exist_ok=True)

    with open(output_filename, 'w') as file:
        # Write image header
        file.write(f"{file_format}\n")
        file.write(f"# Created by IrfanView\n")
        file.write(f"{image_np.shape[1]} {image_np.shape[0]}\n")  # Width and height
        file.write(f"{max_value}\n")  # Maximum pixel value

        # Write pixel values
        for row in image_np:
            if file_format == "P2":  # Grayscale image (PGM)
                file.write(" ".join(map(str, row)) + " \n")
            elif file_format == "P3":  # RGB image (PPM)
                for pixel in row:
                    file.write(" ".join(map(str, pixel)) + " \n")

    print(f"Processed image saved as: {output_filename}")


In [ ]:
# Seam carving functions

import numpy as np

def calculate_energy(image):
    """Calculate energy of each pixel in the image with edge handling."""
    rows, cols = image.shape[0], image.shape[1]
    energy = np.zeros_like(image)

    for y in range(0, rows):
        for x in range(0, cols):
            energy[y][x] = abs(image[y,x] - image[y-1 if y-1>=0 else 0,x]) + \
                           abs(image[y,x] - image[y+1 if y+1<rows else rows-1,x]) + \
                           abs(image[y,x] - image[y,x-1 if x-1>=0 else 0]) + \
                           abs(image[y,x] - image[y,x+1 if x+1<cols else cols-1])
    return energy

def calculate_cumulative_energy(energy):
    """Calculate the cumulative energy matrix for a given energy matrix using dynamic programming."""
    rows, cols = energy.shape[0], energy.shape[1]
    cumulative_energy = np.zeros_like(energy)

    # Copy the first row of energy into cumulative_energy
    cumulative_energy[0, :] = energy[0, :]

    # Fill in the rest of the cumulative energy matrix
    for y in range(1, rows): # Start from the second row
        for x in range(0, cols):
            cumulative_energy[y][x] = min(energy[y][x] + cumulative_energy[y-1][x-1] if x-1>=0 else float('inf'),
                                          energy[y][x] + cumulative_energy[y-1][x],
                                          energy[y][x] + cumulative_energy[y-1][x+1] if x+1<cols else float('inf'))
    return cumulative_energy

def seam_carve(image):
    """Perform seam carving on the image."""
    energy = calculate_energy(image)
    cumulative_energy = calculate_cumulative_energy(energy)

    rows, cols = image.shape[0], image.shape[1]
    min_value_x = np.argmin(cumulative_energy[-1, :])

    # Initialize seam path to store the indices of pixels to be removed
    seam_path = []

    # Trace back to find the seam path
    for y in range(rows - 1, -1, -1):
        seam_path.append((y, min_value_x))

        if y > 0:
            # Determine x indices and values of pixels in the row above
            neighbors = [
                (min_value_x - 1,   cumulative_energy[y-1][min_value_x - 1] if min_value_x - 1 >= 0     else float('inf')),
                (min_value_x,       cumulative_energy[y-1][min_value_x]),
                (min_value_x + 1,   cumulative_energy[y-1][min_value_x + 1] if min_value_x + 1 < cols   else float('inf'))
            ]

            # Find the minimum value and corresponding x index
            min_value_x = min(neighbors, key=lambda x: x[1])[0]


    # Create a copy of the image without the pixels in the seam path
    if len(image.shape) == 2:  # Grayscale image (2D)
        carved_image = np.zeros((rows, cols - 1), dtype=image.dtype)
    elif len(image.shape) == 3 and image.shape[2] == 3:  # RGB image (3D)
        carved_image = np.zeros((rows, cols - 1, 3), dtype=image.dtype)
    else:
        raise ValueError("Unsupported image shape")

    for y, x in seam_path:
        # Exclude the pixel at (y, x) from the image
        if len(image.shape) == 2:  # Grayscale image (2D)
            carved_image[y, :] = np.delete(image[y, :], x, axis=0)
        elif len(image.shape) == 3 and image.shape[2] == 3:  # RGB image (3D)
            carved_image[y, :, :] = np.delete(image[y, :, :], x, axis=0)
        else:
            raise ValueError("Unsupported image shape")

    return carved_image


In [ ]:
# Image processing functions

def process_image(image_filepath, vertical_seams, horizontal_seams=0):
    image_np, max_value = load_image_as_numpy_array(image_filepath)

    # Vertical seams
    for _ in range(vertical_seams):
        image_np = seam_carve(image_np)

    if horizontal_seams != 0:
        image_np = image_np.T  # Transpose for horizontal seam carving

        # Horizontal seams
        for _ in range(horizontal_seams):
            image_np = seam_carve(image_np)

        image_np = image_np.T  # Transpose back to original orientation

    save_image(image_filepath, image_np, max_value, vertical_seams, horizontal_seams)

In [ ]:
# Part I : vertical seam removal - pgm

# For Testing
image_filepath = './test/raw/CAS.pgm'
vertical_seams = 20

# For Grading
# image_filepath = filename
# vertical_seams = verticalSeam2Remove

# Entry Point for Part I
process_image(image_filepath, vertical_seams)


In [ ]:
# Part II : both vertical and horizontal seams removal - pgm

# For Testing
image_filepath = './test/raw/CAS_2.pgm'
vertical_seams = 100
horizontal_seams = 50

# For Grading
# image_filepath = filename
# vertical_seams = verticalSeam2Remove
# horizontal_seams = horizontalSeam2Remove

# Entry Point for Part II
process_image(image_filepath, vertical_seams, horizontal_seams)

In [ ]:
# Part III : both vertical and horizontal seams removal - ppm

# For Testing
filenameG = "./test/raw/flower.ppm"
vSeam2Remove = 300
hSeam2Remove = 100

# For Grading
# filenameG = "flower.ppm"
# vSeam2Remove = 20
# hSeam2Remove = 25

# Entry Point for Part III
process_image(filenameG, vSeam2Remove, hSeam2Remove)

In [ ]:
# Test cell
import filecmp
import os

def test_and_compare(input_files, correct_files):
    results = []

    for my_file, correct_file in zip(input_files, correct_files):
        my_file_stripped = add_suffix_to_filename(my_file, '_stripped')
        correct_file_stripped = add_suffix_to_filename(correct_file, '_stripped')

        strip_trailing_whitespace(my_file, my_file_stripped)
        strip_trailing_whitespace(correct_file, correct_file_stripped)

        same = filecmp.cmp(my_file_stripped, correct_file_stripped, shallow=False)
        results.append("Same" if same else "DIFFERENT")

    return results

def add_suffix_to_filename(filename, suffix):
    base, ext = os.path.splitext(filename)
    return base + suffix + ext

# Define test cases
input_files = [
    "./test/experimental/CAS_processed_20_0.pgm",
    "./test/experimental/CAS_2_processed_100_50.pgm",
    # "./test/experimental/flower_processed_300_100.ppm"
]

correct_files = [
    "./test/expected/CAS_processed_20_0.pgm",
    "./test/expected/CAS_2_processed_100_50.pgm",
    "./test/expected/flower_processed_300_100.ppm"
]

# Run tests and display results
results = test_and_compare(input_files, correct_files)

for idx, result in enumerate(results, 1):
    print(f"Testing part {idx}:")
    print(result)